<!-- # Data Exploration -->


In [1]:
# Built-in library
from pathlib import Path
import re
import json
from typing import Any, Literal, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
import numpy.typing as npt
from pprint import pprint
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)

In [4]:
go_up_from_current_directory(go_up=2)

from QA_and_RAG import PACKAGE_ROOT_PATH
from QA_and_RAG.src.utils.utilities import ProcessFiles
from config import config, settings

/Users/neidu/Desktop/Projects/Personal/My_Projects/Gen-AI-Projects


<!-- ### SQL Database(s) -->

In [5]:
go_up_from_current_directory(go_up=1)


from src.db_utils import SQLFromTabularData
from src.chatbot import Chatbot

/Users/neidu/Desktop/Projects/Personal/My_Projects/Gen-AI-Projects/QA_and_RAG


In [6]:
create_db: SQLFromTabularData = SQLFromTabularData(
    file_path="../data/flat_files/titanic-data.csv",
    db_path="../data/flat_files/stored_data.db",
    table_name="titanic",
)


# create_db: SQLFromTabularData = SQLFromTabularData(
#     file_path="../data/flat_files/titanic-data.csv",
#     db_path="../data/flat_files/stored_data.db",
#     table_name="titanic",
# )


create_db.run()

Error creating/updating the DB: Table 'titanic' already exists.
DB Path: sqlite:///../data/flat_files/stored_data.db
Available table Names: ['breast_cancer', 'diabetes', 'titanic']


In [7]:
titanic_data: pl.DataFrame = pl.read_csv("../data/flat_files/titanic-data.csv")
titanic_data.head()

pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
i64,i64,str,str,f64,i64,i64,str,f64,str,str,str,i64,str
1,1,"""Allen, Miss. Elisabeth Walton""","""female""",29.0,0,0,"""24160""",211.3375,"""B5""","""S""","""2""",null,"""St Louis, MO"""
1,1,"""Allison, Master. Hudson Trevor""","""male""",0.9167,1,2,"""113781""",151.55,"""C22 C26""","""S""","""11""",null,"""Montreal, PQ / Chesterville, ON"""
1,0,"""Allison, Miss. Helen Loraine""","""female""",2.0,1,2,"""113781""",151.55,"""C22 C26""","""S""",null,null,"""Montreal, PQ / Chesterville, ON"""
1,0,"""Allison, Mr. Hudson Joshua Creighton""","""male""",30.0,1,2,"""113781""",151.55,"""C22 C26""","""S""",null,135,"""Montreal, PQ / Chesterville, ON"""
1,0,"""Allison, Mrs. Hudson J C (Bessie Waldo Daniels)""","""female""",25.0,1,2,"""113781""",151.55,"""C22 C26""","""S""",null,null,"""Montreal, PQ / Chesterville, ON"""


In [12]:
query: str = "How many men were in the Titanic?"

titanic_data.filter(pl.col("sex").eq("male")).select(["name"]).shape[0]

843

In [13]:
query: str = "How many women survived the Titanic?"

titanic_data.filter(((pl.col("sex").eq("female")) & (pl.col("survived").eq(1)))).select(
    ["name"]
).shape[0]

339

### Diabetes Dataset

In [14]:
diabetes_data: pl.DataFrame = pl.read_csv("../data/flat_files/diabetes.csv")
diabetes_data.head()

Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
i64,i64,i64,i64,i64,f64,f64,i64,i64
6,148,72,35,0,33.6,0.627,50,1
1,85,66,29,0,26.6,0.351,31,0
8,183,64,0,0,23.3,0.672,32,1
1,89,66,23,94,28.1,0.167,21,0
0,137,40,35,168,43.1,2.288,33,1


In [17]:
query: str = "What is the average age of people with diabetes?"

diabetes_data.filter(pl.col("Outcome").eq(1)).select(
    ["Age"]
).mean().to_series().to_list()[0]

37.06716417910448

In [ ]:
print(diabetes_data["Pregnancies"].unique().to_list())

query: str = "What is the average blood pressure of pregnant people?"
diabetes_data.filter(pl.col("Pregnancies").ge(1)).select(
    ["BloodPressure"]
).mean().to_series().to_list()[0]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17]


69.43531202435312

In [ ]:
from sqlalchemy import create_engine


db_path: str = "../data/sql/chinook.db"
conn = create_engine(f"sqlite:///{db_path}")
query: str = "SELECT name FROM sqlite_master WHERE type='table';"
pl.read_database(query=query, connection=conn.connect()).to_series().to_list()

In [ ]:
query = "SELECT * FROM Track LIMIT 10"

pl.read_database(query=query, connection=conn.connect())

In [ ]:
query = "SELECT * FROM Artist LIMIT 10"

pl.read_database(query=query, connection=conn.connect())

In [ ]:
fp: str = "../data/flat_files/breast-cancer.csv"
df: pl.DataFrame = pl.read_csv(fp)

df.head()